In [1]:
import numpy as np
import pandas as pd

from custom_functions import custom_accuracy_score, custom_confusion_matrix, custom_classification_report

In [2]:
# Load the training dataset
train_csv_path = './archive/sign_mnist_train.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing dataset
test_csv_path = './archive/sign_mnist_test.csv'
test_data = pd.read_csv(test_csv_path)

In [3]:
# Preprocess the data
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:
# Preprocessing function
def preprocess_sign_mnist(train_data, test_data):
    # Separate features and labels
    X_train = train_data.drop('label', axis=1).values
    y_train = train_data['label'].values

    X_test = test_data.drop('label', axis=1).values
    y_test = test_data['label'].values

    # Normalize pixel values to [0, 1]
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    print("Unique training classes:", np.unique(y_train))
    print("Unique testing classes:", np.unique(y_test))

    return X_train, X_test, y_train, y_test

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_sign_mnist(train_data, test_data)

Unique training classes: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
Unique testing classes: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [5]:
class CustomSVM:
    def __init__(self, C=1, kernel='linear', degree=3, gamma='auto'):
        """
        Initialize the Custom SVM Classifier.
        
        Parameters:
        - C (float): Penalty parameter of the error term.
        - kernel (str): Kernel function to use ('linear', 'poly', 'rbf').
        - degree (int): Degree of the polynomial kernel.
        - gamma (str or float): Kernel coefficient for 'rbf' and 'poly' kernels.
        """
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None

    def _kernel(self, X1, X2):
        """
        Compute the kernel function between two sets of samples.
        
        Parameters:
        - X1 (numpy.ndarray): First set of samples.
        - X2 (numpy.ndarray): Second set of samples.
        
        Returns:
        - Kernel matrix.
        """
        if self.kernel == 'linear':
            return np.dot(X1, X2.T)
        elif self.kernel == 'poly':
            return (1 + np.dot(X1, X2.T)) ** self.degree
        elif self.kernel == 'rbf':
            if self.gamma == 'auto':
                self.gamma = 1 / X1.shape[1]
            return np.exp(-self.gamma * np.sum((X1[:, None] - X2) ** 2, axis=-1))

    def fit(self, X, y):
        """
        Train the Custom SVM Classifier.
        
        Parameters:
        - X (numpy.ndarray): Training samples.
        - y (numpy.ndarray): Training labels.
        """
        n_samples, n_features = X.shape
        
        # Initialize Lagrange multipliers
        self.alpha = np.zeros(n_samples)
        
        # Compute kernel matrix
        K = self._kernel(X, X)
        
        # Optimize Lagrange multipliers
        for _ in range(1000):
            for i in range(n_samples):
                grad = np.sum(self.alpha * y * K[:, i]) - 1
                if (y[i] * grad < -1e-5) and (self.alpha[i] < self.C):
                    self.alpha[i] += y[i]
        
        # Identify support vectors
        self.support_vectors = X[self.alpha > 1e-5]
        self.support_vector_labels = y[self.alpha > 1e-5]
        
        # Compute intercept
        self.intercept = np.mean(self.support_vector_labels -
                                np.dot(self._kernel(self.support_vectors, self.support_vectors),
                                       self.support_vector_labels * self.alpha[self.alpha > 1e-5]))

    def predict(self, X):
        """
        Make predictions using the trained Custom SVM Classifier.
        
        Parameters:
        - X (numpy.ndarray): Samples to predict.
        
        Returns:
        - Predicted labels.
        """
        decision_values = np.dot(self._kernel(X, self.support_vectors),
                                self.support_vector_labels * self.alpha[self.alpha > 1e-5]) + self.intercept
        return np.sign(decision_values)

In [ ]:
# Initialize and train the Custom SVM model
svm = CustomSVM(C=1, kernel='linear', gamma='auto', degree=3)  # Set appropriate parameters
svm.fit(X_train, y_train)  # Train the model

# Predict on the test set
y_pred = svm.predict(X_test)

# Evaluation metrics using custom implementations
accuracy = custom_accuracy_score(y_test, y_pred)
print(f"Accuracy of the model calculated : {accuracy:.2f}")
custom_confusion_matrix(y_true=y_test, y_pred=y_pred, num_classes=len(np.unique(y_train)))
custom_classification_report(y_test, y_pred, num_classes=len(np.unique(y_train)))



In [ ]:
def hypertune_custom_svm(X_train, y_train, X_test, y_test, param_grid):
    """
    Hypertune the parameters of the Custom SVM implementation with progress updates.
    
    Parameters:
        X_train, y_train: Training data and labels
        X_test, y_test: Testing data and labels
        param_grid: Dictionary of parameters to hypertune
        
    Returns:
        best_params: Best combination of parameters
        best_accuracy: Highest accuracy achieved
    """
    best_params = None
    best_accuracy = 0
    results = []
    total_combinations = (len(param_grid['C']) *
                          len(param_grid['kernel']) *
                          len(param_grid['gamma']) *
                          len(param_grid['degree']))
    progress = 0  # Track the current progress

    print(f"Total combinations to test: {total_combinations}\n")

    for C in param_grid['C']:
        for kernel in param_grid['kernel']:
            for gamma in param_grid['gamma']:
                for degree in param_grid['degree']:
                    progress += 1
                    print(f"Testing combination {progress}/{total_combinations}: C={C}, kernel={kernel}, gamma={gamma}, degree={degree}")
                    
                    # Train the Custom SVM
                    svm = CustomSVM(C=C, kernel=kernel, degree=degree, gamma=gamma)
                    svm.fit(X_train, y_train)
                    
                    # Predict on the test set
                    y_pred = svm.predict(X_test)
                    
                    # Evaluate accuracy
                    accuracy = custom_accuracy_score(y_test, y_pred)
                    print(f"Accuracy for this combination: {accuracy:.4f}\n")
                    
                    # Store results
                    results.append((C, kernel, gamma, degree, accuracy))
                    
                    # Update best parameters if current is better
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = {'C': C, 'kernel': kernel, 'gamma': gamma, 'degree': degree}

                    # Print progress every 10% of combinations
                    if progress % (total_combinations // 10) == 0 or progress == total_combinations:
                        print(f"Progress: {progress}/{total_combinations} combinations tested.")
    
    print("\nBest Parameters Found:")
    print(best_params)
    print(f"Best Accuracy: {best_accuracy:.4f}")
    
    return best_params, best_accuracy, results
